mtask处理代码
0. 得到最基础的mtask_relative，会有识别不出来的情况
通过运行改版后的train.py，并且要把lerobot_dataset里面


In [1]:
# 把train生成的pure文件转移到meta里面,然后修改parent_file
parent_file="/home/zhonglinyue/code/lerobot_diy/training_dataset/0905/push/meta/"
pure_file=parent_file+"modified_tasks_pure.jsonl"
modified_file=parent_file+"full_modified_tasks.jsonl"
relative_file=parent_file+"mtask_relative.jsonl"
grid_5cm=parent_file+"mtask_5cm.jsonl"
grid_2cm=parent_file+"mtask_2cm.jsonl"
objects=["router","sticker"]
removr_same=parent_file+"filtered_full_modified_tasks.jsonl"


1. 补完原始的数值,但是会产生重复的.

In [2]:
import json
import re
from pathlib import Path

def parse_coords(text, key):
    """
    自动识别 2D pixel (x, y) 或 3D (x, y, z) 坐标
    返回 tuple(float, float) 或 tuple(float, float, float)
    """
    # 3D 坐标匹配
    pattern_3d = rf"{key} at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)"
    match = re.search(pattern_3d, text)
    if match:
        return tuple(map(float, match.groups()))
    
    # 2D 坐标匹配
    pattern_2d = rf"{key} at pixel \((-?\d+\.?\d*), (-?\d+\.?\d*)\)"
    match = re.search(pattern_2d, text)
    if match:
        return tuple(map(float, match.groups()))
    
    return None

def replace_coords(text, key, coords):
    if coords is None:
        return text
    
    # 3D 坐标
    if len(coords) == 3:
        pattern = rf"{key} at \([^)]*\)"
        repl = f"{key} at ({coords[0]:.3f}, {coords[1]:.3f}, {coords[2]:.3f})"
    # 2D pixel 坐标
    elif len(coords) == 2:
        pattern = rf"{key} at pixel \([^)]*\)"
        repl = f"{key} at pixel ({coords[0]:.1f}, {coords[1]:.1f})"
    else:
        return text

    if re.search(pattern, text):
        return re.sub(pattern, repl, text)
    else:
        if "|" in text:
            return text.strip() + " | " + repl
        else:
            return text.strip() + " | " + repl

def linear_interpolate(start, end, n):
    """n = gap size (包含两端，所以生成 n-1 个点)"""
    if start is None or end is None:
        return [None] * (n - 1)
    result = []
    for i in range(1, n):
        frac = i / n
        interp = tuple(s + (e - s) * frac for s, e in zip(start, end))
        result.append(interp)
    return result

def process_file(input_file, output_file, objects):
    lines = [json.loads(line) for line in Path(input_file).read_text().splitlines()]
    modified = 0

    episodes = {}
    for entry in lines:
        episodes.setdefault(entry["episode_index"], []).append(entry)

    for ep, entries in episodes.items():
        entries.sort(key=lambda x: x["frame_index"])

        for obj in objects:
            coords_list = [parse_coords(e["task"], obj) for e in entries]

            # 补全开头缺失
            first_valid = next((c for c in coords_list if c is not None), None)
            if first_valid:
                for i in range(len(coords_list)):
                    if coords_list[i] is None:
                        coords_list[i] = first_valid
                        modified += 1
                    else:
                        break

            # 补全结尾缺失
            last_valid = next((c for c in reversed(coords_list) if c is not None), None)
            if last_valid:
                for i in range(len(coords_list)-1, -1, -1):
                    if coords_list[i] is None:
                        coords_list[i] = last_valid
                        modified += 1
                    else:
                        break

            # 插值中间缺失
            i = 0
            while i < len(coords_list):
                if coords_list[i] is None:
                    j = i
                    while j < len(coords_list) and coords_list[j] is None:
                        j += 1
                    if i > 0 and j < len(coords_list):
                        interp_vals = linear_interpolate(coords_list[i-1], coords_list[j], j-i+1)
                        for k in range(i, j):
                            coords_list[k] = interp_vals[k-i]
                            modified += 1
                    i = j
                else:
                    i += 1

            # 更新 task 文本
            for idx, e in enumerate(entries):
                e["task"] = replace_coords(e["task"], obj, coords_list[idx])

    with open(output_file, "w") as f:
        for ep in sorted(episodes):
            for entry in sorted(episodes[ep], key=lambda x: x["frame_index"]):
                f.write(json.dumps(entry, ensure_ascii=False) + "\n")

    print(f"修改成功: {modified}")

if __name__ == "__main__":
    objects_all = ["gripper"] + objects
    # 修改这里即可
    process_file(pure_file, modified_file, objects=objects_all)


修改成功: 19744


2. 去掉重复的

In [3]:
import json
import re
from pathlib import Path
from collections import defaultdict

def parse_coords(text):
    """解析 task 里的所有物体坐标，返回 {obj: (x,y,z)}"""
    pattern = r"(\w+) at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)"
    matches = re.findall(pattern, text)
    coords = {}
    for key, x, y, z in matches:
        coords[key] = (float(x), float(y), float(z))
    return coords

def replace_all_coords(task, avg_coords):
    """把 task 里所有的物体位置替换成平均后的值"""
    for key, (x, y, z) in avg_coords.items():
        pattern = rf"{key} at \([^)]*\)"
        repl = f"{key} at ({x:.3f}, {y:.3f}, {z:.3f})"
        task = re.sub(pattern, repl, task)
    return task

def average_dicts(coord_list):
    """对一批 {obj: (x,y,z)} 求平均"""
    if not coord_list:
        return {}
    sums = defaultdict(lambda: [0.0, 0.0, 0.0])
    counts = defaultdict(int)
    for coords in coord_list:
        for key, (x, y, z) in coords.items():
            sums[key][0] += x
            sums[key][1] += y
            sums[key][2] += z
            counts[key] += 1
    avg = {}
    for key in sums:
        avg[key] = (
            sums[key][0] / counts[key],
            sums[key][1] / counts[key],
            sums[key][2] / counts[key],
        )
    return avg

def merge_duplicates(input_file, output_file):
    lines = [json.loads(line) for line in Path(input_file).read_text().splitlines()]

    # 按 (episode_index, frame_index) 分组
    grouped = defaultdict(list)
    for entry in lines:
        key = (entry["episode_index"], entry["frame_index"])
        grouped[key].append(entry)

    merged = []
    for (ep, fr), entries in grouped.items():
        # 解析所有 task 坐标
        coords_list = [parse_coords(e["task"]) for e in entries]
        avg_coords = average_dicts(coords_list)

        # 用第一条作为基准，替换坐标
        base = entries[0].copy()
        base["task"] = replace_all_coords(base["task"], avg_coords)
        merged.append(base)

    # 按 episode, frame 排序输出
    merged.sort(key=lambda x: (x["episode_index"], x["frame_index"]))
    with open(output_file, "w") as f:
        for entry in merged:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

    print(f"合并完成: {len(merged)} 行")

if __name__ == "__main__":   # 输入文件   # 输出文件
    merge_duplicates(modified_file, removr_same)


合并完成: 42582 行


3. 检查

In [5]:
# 检查
import json
import re
from pathlib import Path

def parse_coords(text, key):
    # 支持两种格式： "... at pixel (x, y)" 或 "... relative to gripper is (x, y)"
    patterns = [
        rf"{key} at pixel \((-?\d+\.?\d*), (-?\d+\.?\d*)\)",
        rf"{key} relative to gripper is \((-?\d+\.?\d*), (-?\d+\.?\d*)\)",
        rf"{key} at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)"  # 备用三维
    ]
    for pat in patterns:
        if re.search(pat, text):
            return True
    return False


def check_file(input_file, objects):
    lines = [json.loads(line) for line in Path(input_file).read_text().splitlines()]
    missing = []

    for entry in lines:
        task = entry["task"]
        frame_missing = [obj for obj in objects if not parse_coords(task, obj)]
        if frame_missing:
            missing.append({
                "episode_index": entry["episode_index"],
                "frame_index": entry["frame_index"],
                "missing": frame_missing,
                "task": task
            })

    if missing:
        print(f"⚠️ 有 {len(missing)} 个 task 缺少物体坐标：")
        for m in missing[:20]:  # 只打印前20个，避免太长
            print(f"  ep{m['episode_index']} frame{m['frame_index']} 缺失: {m['missing']}")
    else:
        print("✅ 所有 task 都包含了完整的物体坐标！")

if __name__ == "__main__":
    objects_all = ["gripper"]+objects  # 需要检查的物体
    # file=parent_file+"fixed_tasks.jsonl"
    check_file(removr_same, objects_all)


✅ 所有 task 都包含了完整的物体坐标！


In [10]:
import json
from pathlib import Path

# parent_file = "./"  # 你自己的路径
input_file = parent_file + "filtered_full_modified_tasks.jsonl"
output_file = parent_file + "fixed_tasks.jsonl"

bad_lines = 0
fixed_count = 0

with open(input_file, "r") as f_in, open(output_file, "w") as f_out:
    for i, line in enumerate(f_in, 1):
        line = line.strip()
        if not line:
            continue  # 跳过空行
        try:
            data = json.loads(line)
        except json.JSONDecodeError as e:
            print(f"⚠️ 第 {i} 行不是合法 JSON，跳过: {e}")
            bad_lines += 1
            continue

        task = data["task"]

        # 如果缺少 green，就补上
        if "green at pixel" not in task:
            task = task.strip() + " | green at pixel (45.0, 52.0)"
            data["task"] = task
            fixed_count += 1

        f_out.write(json.dumps(data, ensure_ascii=False) + "\n")

print(f"✅ 处理完成，结果保存在 {output_file}")
print(f"补全 green 的数量: {fixed_count}")
print(f"跳过非法行数量: {bad_lines}")


✅ 处理完成，结果保存在 /home/zhonglinyue/code/lerobot_diy/training_dataset/pusht/meta/fixed_tasks.jsonl
补全 green 的数量: 144
跳过非法行数量: 0


支持pusht仿真环境，应该返回的是像素值

In [13]:
import json
import re
from pathlib import Path

def parse_coords(text, key):
    """
    解析 2D pixel 坐标: key at pixel (x, y)
    """
    pattern = rf"{key} at pixel \((-?\d+\.?\d*), (-?\d+\.?\d*)\)"
    match = re.search(pattern, text)
    if match:
        return tuple(map(float, match.groups()))
    return None

def clean_task(text):
    """
    去掉原 task 中的 '| gripper at pixel ... | accessory at pixel ... | router at pixel ...' 坐标信息
    """
    return re.sub(r"\|.*?at pixel \((-?\d+\.?\d*), (-?\d+\.?\d*)\)", "", text).strip()

def process_file(input_file, output_file, objects):
    lines = [json.loads(line) for line in Path(input_file).read_text().splitlines()]

    with open(output_file, "w") as f:
        for entry in lines:
            original_task = clean_task(entry["task"])
            g = parse_coords(entry["task"], "gripper")

            rel_parts = []
            if g:
                for obj in objects:
                    o = parse_coords(entry["task"], obj)
                    if o:
                        # 计算2D相对坐标
                        rel = (o[0] - g[0], o[1] - g[1])
                        rel_parts.append(
                            f"{obj} position relative to gripper is ({rel[0]:.1f}, {rel[1]:.1f})"
                        )

            # 拼接最终 task
            if rel_parts:
                entry["task"] = original_task + ", " + ", ".join(rel_parts)
            else:
                entry["task"] = original_task

            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

if __name__ == "__main__":
    file=parent_file+"fixed_tasks.jsonl"
    process_file(file, relative_file, objects)


In [15]:
# pusht直接用这个
import json
import re
from pathlib import Path

def parse_coords(text, key):
    """
    解析 2D pixel 坐标: key at pixel (x, y)
    """
    pattern = rf"{key} at pixel \((-?\d+\.?\d*), (-?\d+\.?\d*)\)"
    match = re.search(pattern, text)
    if match:
        return tuple(map(float, match.groups()))
    return None

def clean_task(text):
    """
    去掉原 task 中的 '| xxx at pixel ...' 坐标信息
    """
    return re.sub(r"\|.*?at pixel \((-?\d+\.?\d*), (-?\d+\.?\d*)\)", "", text).strip()

def format_coord(value: float) -> str:
    """
    四舍五入为整数，并格式化为至少3位，正数补0，负数保留负号后补0
    """
    rounded = int(round(value))
    if rounded >= 0:
        return f"{rounded:03d}"
    else:
        return f"-{abs(rounded):03d}"

def process_file(input_file, output_file, objects):
    lines = [json.loads(line) for line in Path(input_file).read_text().splitlines()]

    with open(output_file, "w") as f:
        for entry in lines:
            original_task = clean_task(entry["task"])
            g = parse_coords(entry["task"], "gripper")

            rel_parts = []
            if g:
                for obj in objects:
                    o = parse_coords(entry["task"], obj)
                    if o:
                        # 计算2D相对坐标
                        rel = (o[0] - g[0], o[1] - g[1])
                        rel_parts.append(
                            f"{obj} position relative to gripper is ({format_coord(rel[0])}, {format_coord(rel[1])})"
                        )

            # 拼接最终 task
            if rel_parts:
                entry["task"] = original_task + ", " + ", ".join(rel_parts)
            else:
                entry["task"] = original_task

            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

if __name__ == "__main__":
    file = parent_file + "fixed_tasks.jsonl"
    process_file(file, relative_file, objects)


变成mtask_relative

In [11]:
import json
import re
from pathlib import Path

def parse_coords(text, key):
    pattern = rf"{key} at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)"
    match = re.search(pattern, text)
    if match:
        return tuple(map(float, match.groups()))
    return None

def clean_task(text):
    """
    去掉原 task 中的 '| gripper at ... | accessory at ... | router at ...' 坐标信息
    """
    return re.sub(r"\|.*?at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)", "", text).strip()

def process_file(input_file, output_file, objects):
    lines = [json.loads(line) for line in Path(input_file).read_text().splitlines()]

    with open(output_file, "w") as f:
        for entry in lines:
            original_task = clean_task(entry["task"])
            g = parse_coords(entry["task"], "gripper")

            rel_parts = []
            if g:
                for obj in objects:
                    o = parse_coords(entry["task"], obj)
                    if o:
                        rel = tuple(o_i - g_i for o_i, g_i in zip(o, g))
                        rel_parts.append(
                            f"{obj} position relative to gripper is ({rel[0]:.3f}, {rel[1]:.3f}, {rel[2]:.3f})"
                        )

            # 拼接最终 task
            if rel_parts:
                entry["task"] = original_task + ", " + ", ".join(rel_parts)
            else:
                entry["task"] = original_task

            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

if __name__ == "__main__":
    process_file(removr_same, relative_file, objects)


5cm格子

In [12]:
import json
import re
from pathlib import Path

def parse_coords(text, key):
    pattern = rf"{key} at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)"
    match = re.search(pattern, text)
    if match:
        return tuple(map(float, match.groups()))
    return None

def clean_task(text):
    """
    去掉原 task 中的 '| gripper at ... | accessory at ... | router at ...' 坐标信息
    """
    return re.sub(r"\|.*?at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)", "", text).strip()

def process_file(input_file, output_file, objects):
    lines = [json.loads(line) for line in Path(input_file).read_text().splitlines()]

    with open(output_file, "w") as f:
        for entry in lines:
            original_task = clean_task(entry["task"])
            g = parse_coords(entry["task"], "gripper")

            rel_parts = []
            if g:
                for obj in objects:
                    o = parse_coords(entry["task"], obj)
                    if o:
                        # 计算相对 gripper 的坐标
                        rel = tuple(o_i - g_i for o_i, g_i in zip(o, g))
                        # 转换到 5cm 格子并取整
                        grid = tuple(round(val / 0.05) for val in rel)
                        # 格式化为两位整数
                        grid_str = tuple(f"{num:02d}" if num >= 0 else f"-{abs(num):02d}" for num in grid)
                        rel_parts.append(
                            f"{obj} position relative to gripper is ({grid_str[0]}, {grid_str[1]}, {grid_str[2]}) in 5cm grid units"
                        )

            # 拼接最终 task
            if rel_parts:
                entry["task"] = original_task + ", " + ", ".join(rel_parts)
            else:
                entry["task"] = original_task

            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

if __name__ == "__main__":
    process_file(removr_same, grid_5cm, objects)


# 2cm

In [13]:
import json
import re
from pathlib import Path

def parse_coords(text, key):
    pattern = rf"{key} at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)"
    match = re.search(pattern, text)
    if match:
        return tuple(map(float, match.groups()))
    return None

def clean_task(text):
    """
    去掉原 task 中的 '| gripper at ... | accessory at ... | router at ...' 坐标信息
    """
    return re.sub(r"\|.*?at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)", "", text).strip()

def process_file_2cm(input_file, output_file, objects):
    lines = [json.loads(line) for line in Path(input_file).read_text().splitlines()]

    with open(output_file, "w") as f:
        for entry in lines:
            original_task = clean_task(entry["task"])
            g = parse_coords(entry["task"], "gripper")

            rel_parts = []
            if g:
                for obj in objects:
                    o = parse_coords(entry["task"], obj)
                    if o:
                        # 计算相对 gripper 的坐标
                        rel = tuple(o_i - g_i for o_i, g_i in zip(o, g))
                        # 转换到 2cm 格子并取整
                        grid = tuple(round(val / 0.02) for val in rel)
                        # 格式化为两位整数
                        grid_str = tuple(f"{num:02d}" if num >= 0 else f"-{abs(num):02d}" for num in grid)
                        rel_parts.append(
                            f"{obj} position relative to gripper is ({grid_str[0]}, {grid_str[1]}, {grid_str[2]}) in 2cm grid units"
                        )

            # 拼接最终 task
            if rel_parts:
                entry["task"] = original_task + ", " + ", ".join(rel_parts)
            else:
                entry["task"] = original_task

            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

if __name__ == "__main__":
    process_file_2cm(removr_same, grid_2cm, objects)
